In [1]:
import cv2
import numpy as np
import tkinter as tk
from tkinter import filedialog, Label, Button
from PIL import Image, ImageTk

def calculate_shape_features(contour): # Calculate shape features: circularity, aspect ratio, solidity """
    area = cv2.contourArea(contour)
    perimeter = cv2.arcLength(contour, True)
    x, y, w, h = cv2.boundingRect(contour)
    
    circularity = (4 * np.pi * area) / (perimeter ** 2) if perimeter > 0 else 0  # Circularity
    aspect_ratio = w / h if h > 0 else 0  # Aspect ratio
    hull = cv2.convexHull(contour)  # Convex hull
    hull_area = cv2.contourArea(hull)  # Convex hull area
    solidity = area / hull_area if hull_area > 0 else 0  # Solidity
    
    return circularity, aspect_ratio, solidity, (x, y, w, h)

def classify_shape(circularity, aspect_ratio, solidity): # Classify the shape based on calculated features """
    if circularity > 0.8 and 0.9 < aspect_ratio < 1.1:
        return "Circle"
    elif aspect_ratio > 1.5:
        return "Ellipse"
    elif 0.95 <= aspect_ratio <= 1.05 and solidity > 0.95:
        return "Square"
    elif solidity < 0.9:
        return "Irregular"
    else:
        return "Polygon"

def detect_and_label_shapes(image_path):# Process image, detect multiple shapes, and label them on the image """
    img = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
    _, binary = cv2.threshold(img, 127, 255, cv2.THRESH_BINARY_INV)  # Convert to binary
    contours, _ = cv2.findContours(binary, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)  # Find contours
    
    img_color = cv2.cvtColor(img, cv2.COLOR_GRAY2BGR)  # Convert to BGR for drawing
    result_text = ""  # Store shape classification results
    
    for contour in contours:
        circularity, aspect_ratio, solidity, (x, y, w, h) = calculate_shape_features(contour)
        shape = classify_shape(circularity, aspect_ratio, solidity)
        
        # Draw green bounding box and label
        cv2.drawContours(img_color, [contour], -1, (0, 255, 0), 2)
        cv2.rectangle(img_color, (x, y), (x + w, y + h), (0, 255, 0), 2)
        cv2.putText(img_color, shape, (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 1)
        
        result_text += f"Shape: {shape} | Circularity: {circularity:.2f}, Aspect Ratio: {aspect_ratio:.2f}, Solidity: {solidity:.2f}\n"
    
    return img_color, result_text

def process_image(filepath):# Process uploaded image and display detection results in the GUI """
    img_color, result_text = detect_and_label_shapes(filepath)

    # Update GUI result text
    result_label.config(text=result_text)

    # Convert OpenCV image to Tkinter-compatible format
    img_pil = Image.fromarray(cv2.cvtColor(img_color, cv2.COLOR_BGR2RGB))
    img_pil.thumbnail((300, 300))
    img_tk = ImageTk.PhotoImage(img_pil)

    # Display processed image in GUI
    image_label.config(image=img_tk)
    image_label.image = img_tk

def upload_image():# Allow the user to select and upload an image """
    filepath = filedialog.askopenfilename(filetypes=[("Image files", "*.png *.jpg *.bmp")])
    if filepath:
        image = Image.open(filepath)
        image.thumbnail((300, 300))
        img_tk = ImageTk.PhotoImage(image)
        
        # Display original image in GUI
        image_label.config(image=img_tk)
        image_label.image = img_tk

        # Process and analyze image
        process_image(filepath)

# Initialize GUI
root = tk.Tk()
root.title("SEM Particle Analysis")

Label(root, text="Upload SEM image for analysis:").pack()
Button(root, text="Select Image", command=upload_image).pack()

image_label = Label(root)
image_label.pack()

result_label = Label(root, text="", justify="left")
result_label.pack()

root.mainloop()
